# Dataset Preparation

## Libraries

In [128]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import urllib.request

## Functions

In [129]:
# function to extract html document from given url
def Links_Extraction(url, url_string):
    '''
    Parameters:
        url
        url_string
    Usage: 
        Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'session')
    '''
    
    response = requests.get(url)
    
    html_document = response.text
  
    soup = BeautifulSoup(html_document, 'html.parser')
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(url_string)}):
        links.append(link.get('href'))
        #print(link.get('href'))
    
    return links

In [130]:
# function to prepare dataset by year
def fia_f1_data(season, gp_city, gp_link):
    '''Insert the season = year and grand_prix as a list with fia links'''
    print('Season: ' + str(season) + ' | Source: fia.com')

    dataset = pd.DataFrame()
    n = 0
    for i in gp_link:
        url = f'https://www.fia.com{i}'
        print('Circuit: ' + str(gp_city[n]) + ' | ' + url)
        
        source = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(source,'lxml')

        # classification
        table_class = soup.find_all('table')[0] 
        df_class = pd.read_html(str(table_class), flavor='bs4', header=[1])[0]
        df_class.drop(['Unnamed: 5'],axis=1, inplace=True)
        name = df_class['DRIVER'].str.split(' ',n = 1, expand = True)
        df_class['DRIVER_SNAME'] = name[0].astype(str).str[0] + '. ' + name[1]
        df_class['GRAND_PRIX'] = gp_city[n].capitalize()
        df_class['SEASON'] = season
        df_class = df_class.rename (columns={'POS': 'CLASS', 'TIME': 'RACE_TIME'}) 
        n = n + 1
        # fastest laps
        table_flaps = soup.find_all('table')[1] 
        df_flaps = pd.read_html(str(table_flaps), flavor='bs4', header=[1])[0]
        df_flaps.drop(['Unnamed: 7','Unnamed: 8'],axis=1, inplace=True)
        df_flaps = df_flaps.rename(columns={'POS': 'FLAP_POS', 'LAP TIME': 'FLAP_TIME', 'LAP': 'F_LAP', 'GAP': 'FLAP_GAP', 'KM/H': 'FLAP_KM/H', 'TIME': 'FLAP_TIME'}) 

        # best sector times
        table_bs = soup.find_all('table')[2] 
        df_bs = pd.read_html(str(table_bs), flavor='bs4', header=[2])[0]
        df_bs.drop(['Unnamed: 7'],axis=1, inplace=True)
        df_bs_1 = df_bs[['POS', 'DRIVER',   'TIME']]
        df_bs_1 = df_bs_1.rename(columns={'POS': 'BS1_POS', 'DRIVER':   'BS1_DRIVER', 'TIME':   'BS1_TIME'}) 
        df_bs_2 = df_bs[['POS', 'DRIVER.1', 'TIME.1']]
        df_bs_2 = df_bs_2.rename(columns={'POS': 'BS2_POS', 'DRIVER.1': 'BS2_DRIVER', 'TIME.1': 'BS2_TIME'}) 
        df_bs_3 = df_bs[['POS', 'DRIVER.2', 'TIME.2']]
        df_bs_3 = df_bs_3.rename(columns={'POS': 'BS3_POS', 'DRIVER.2': 'BS3_DRIVER', 'TIME.2': 'BS3_TIME'}) 

        # speed traps
        table_straps = soup.find_all('table')[3] 
        df_straps = pd.read_html(str(table_straps), flavor='bs4', header=[1])[0]
        df_straps.drop(['TEAM','TIME','Unnamed: 5'],axis=1, inplace=True)
        df_straps = df_straps.rename(columns={'POS': 'ST_POS', 'KM/H': 'STKM/H'})

        # maximum speeds
        table_mspeeds = soup.find_all('table')[4] 
        df_mspeeds = pd.read_html(str(table_mspeeds), flavor='bs4', header=[2])[0]
        df_mspeeds.drop(['Unnamed: 7'],axis=1, inplace=True)
        df_mspeeds_1 = df_mspeeds[['POS', 'DRIVER',   'KM/H']]
        df_mspeeds_1 = df_mspeeds_1.rename(columns={'POS': 'I1_POS', 'DRIVER':   'I1_DRIVER', 'KM/H':   'I1_KM/H'}) 
        df_mspeeds_2 = df_mspeeds[['POS', 'DRIVER.1', 'KM/H.1']]
        df_mspeeds_2 = df_mspeeds_2.rename(columns={'POS': 'I2_POS', 'DRIVER.1': 'I2_DRIVER', 'KM/H.2': 'I2_KM/H'}) 
        df_mspeeds_3 = df_mspeeds[['POS', 'DRIVER.2', 'KM/H.2']]
        df_mspeeds_3 = df_mspeeds_3.rename(columns={'POS': 'FL_POS', 'DRIVER.2': 'FL_DRIVER', 'KM/H.2': 'FL_KM/H'}) 

        # pit stops

        table_pstops = soup.find_all('table')[5] 
        df_pstops = pd.read_html(str(table_pstops), flavor='bs4', header=[1])[0]
        df_pstops = df_pstops.rename(columns={'NO': 'DRIVER_NO', 'TOTAL TIME':   'PS_TOTAL_TIME'})
        t = df_pstops.index[df_pstops['DRIVER_NO'] == 'RACE - PIT STOP - DETAIL'].to_list() 
        df_pstops = df_pstops[:t[0]]
        df_pstops = df_pstops[['DRIVER_NO','DRIVER','STOPS','PS_TOTAL_TIME']]


        # merge tables 
        fia_df = pd.merge(df_class, df_flaps, how='left', on=['DRIVER'])

        fia_df = pd.merge(fia_df, df_bs_1,  how='left', left_on='DRIVER_SNAME', right_on='BS1_DRIVER')
        fia_df = pd.merge(fia_df, df_bs_2,  how='left', left_on='DRIVER_SNAME', right_on='BS2_DRIVER')
        fia_df = pd.merge(fia_df, df_bs_3,  how='left', left_on='DRIVER_SNAME', right_on='BS3_DRIVER')
        fia_df.drop(['BS1_DRIVER','BS2_DRIVER','BS3_DRIVER'],axis=1, inplace=True)

        fia_df = pd.merge(fia_df,   df_straps, how='left', on=['DRIVER'])

        fia_df = pd.merge(fia_df, df_mspeeds_1,  how='left', left_on='DRIVER_SNAME', right_on='I1_DRIVER')
        fia_df = pd.merge(fia_df, df_mspeeds_2,  how='left', left_on='DRIVER_SNAME', right_on='I2_DRIVER')
        fia_df = pd.merge(fia_df, df_mspeeds_3,  how='left', left_on='DRIVER_SNAME', right_on='FL_DRIVER')
        fia_df.drop(['I1_DRIVER','I2_DRIVER','FL_DRIVER'],axis=1, inplace=True)

        fia_df = pd.merge(fia_df, df_pstops, how='left', on=['DRIVER'])

        dataset = dataset.append(fia_df)
        
    return dataset

In [131]:
# function to prepare dataset by season - extracts qualification and grid position
def fia_f1_session(season, gp_city, gp_link):
    '''Insert the season = year and grand_prix as a list with fia links'''
    print('Season: ' + str(season) + ' | Source: fia.com')

    dataset = pd.DataFrame()
    n = 0
    for i in gp_link:
        url = f'https://www.fia.com{i}'
        print('Circuit: ' + str(gp_city[n]) + ' | ' + url)
        
        source = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(source,'lxml')    
        
        #qualification
        table_qual = soup.find_all('table')[3] 
        df_qual = pd.read_html(str(table_qual), flavor='bs4', header=[1])[0]
        df_qual = df_qual[['POS','DRIVER','Q1','LAPS','Q2','LAPS.1','Q3','LAPS.2']]
        df_qual = df_qual.rename (columns={'POS': 'QL_CLASS', 'Q1': 'QL_TIME1','Q2': 'QL_TIME2','Q3': 'QL_TIME3',
                                           'LAPS':'QL_LAPS1','LAPS.1':'QL_LAPS2','LAPS.2':'QL_LAPS3'})
        df_qual['GRAND_PRIX'] = gp_city[n].capitalize()
        df_qual['SEASON'] = season

        #grid
        table_grid = soup.find_all('table')[4] 
        df_grid = pd.read_html(str(table_grid), flavor='bs4', header=[1])[0]
        df_grid = df_grid[['POS','DRIVER','TIME']]
        df_grid = df_grid.rename (columns={'POS': 'GD_CLASS', 'TIME': 'GD_TIME'})

        fia_df = pd.merge(df_qual, df_grid, how='left', on=['DRIVER'])
        n = n + 1
        dataset = dataset.append(fia_df)

    return dataset

In [132]:
# grand prix country list taken by date of ocorrence
def f1_gp_circuits(season = 2022):
    '''Extracts the Race Name and GP and ordered by date of occurence'''
    print('Season: ' + str(season) + ' | Source: espn.com')
    url = f'https://www.espn.com/f1/schedule/_/year/{season}'

    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source,'lxml')
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table), flavor='bs4', header=[0])[0]
    df.drop(['Winner/Lights Out','TV'],axis=1, inplace=True)

    gp = df['Race'].str.split(' GP',n = 1, expand = True)
    df['GP'] = gp[0].str.lower()
    # correction on the names that will feed the links
    df['GP'] = df['GP'].str.replace('socar ', '')
    df['GP'] = df['GP'].str.replace('vtb ', '')
    df['GP'] = df['GP'].str.replace('arabian', 'arabia')
    df['GP'] = df['GP'].str.replace('etihad airways ', '')
    df['GP'] = df['GP'].str.replace(' ', '-')
    df['GP'] = df['GP'].str.replace('singapore-airlines-singapore-gmarina-bay-street-circuit', 'singapore')
    df['GP'] = df['GP'].str.replace('mercedes-benz-german', 'german')
    df['GP'] = df['GP'].str.replace('rolex-british','british')
    return df

## Extraction

## 2021

In [133]:
links_race_21 = Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'race-classification')

In [134]:
links_race_21

['/events/fia-formula-one-world-championship/season-2021/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/emilia-romagna-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/portuguese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/spanish-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/french-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/styrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/austrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/british-grand-prix/race-classification',
 '/event

In [135]:
links_session_21 = Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'session')

In [136]:
c_21 = f1_gp_circuits(season = 2021)
city_21 = c_21['GP'].tolist()
c_21

Season: 2021 | Source: espn.com


,Date,Race,GP
0,Mar 26 - 28,Bahrain GPBahrain International Circuit,bahrain
1,Apr 16 - 18,Emilia Romagna GPAutodromo Enzo e Dino Ferrari,emilia-romagna
2,Apr 30 - May 2,Portuguese GPAutódromo Internacional Do Algarve,portuguese
3,May 7 - 9,Spanish GPCircuit de Barcelona-Catalunya,spanish
4,May 20 - 23,Monaco GPCircuit de Monaco,monaco
5,Jun 4 - 6,Socar Azerbaijan GPBaku City Circuit,azerbaijan
6,Jun 18 - 20,French GPCircuit Paul Ricard,french
7,Jun 25 - 27,Austrian GPRed Bull Ring,austrian
8,Jul 2 - 4,Austrian GP 2Red Bull Ring,austrian
9,Jul 16 - 18,British GPSilverstone Circuit,british


In [137]:
df_2021 = fia_f1_data(2021, gp_city = city_21, gp_link = links_race_21)

Season: 2021 | Source: fia.com
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/bahrain-grand-prix/race-classification
Circuit: emilia-romagna | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/emilia-romagna-grand-prix/race-classification
Circuit: portuguese | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/portuguese-grand-prix/race-classification
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/spanish-grand-prix/race-classification
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/monaco-grand-prix/race-classification
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/azerbaijan-grand-prix/race-classification
Circuit: french | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/french-grand-prix/race-classification
Circuit: austrian | htt

In [138]:
df_2021

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,STKM/H,I1_POS,I1_KM/H,I2_POS,KM/H.1,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,Bahrain,2021,4.0,1:34.015,...,315.2,10.0,239.5,2.0,267.7,13.0,290.4,44,2,48.915
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,Bahrain,2021,2.0,1:33.228,...,327.0,15.0,237.9,6.0,266.1,12.0,291.1,33,2,48.615
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,Bahrain,2021,1.0,1:32.090,...,318.9,11.0,239.5,5.0,266.6,16.0,290.0,77,3,1:21.725
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,Bahrain,2021,6.0,1:34.396,...,324.2,16.0,237.6,7.0,263.4,8.0,291.8,4,2,50.539
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,Bahrain,2021,3.0,1:33.970,...,324.7,5.0,240.5,4.0,267.0,17.0,290.0,11,3,1:12.289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,15,Sergio Perez,Red Bull Racing Honda,55,1:24:12.571,S. Perez,Abu-dhabi,2021,2.0,1:26.419,...,318.4,5.0,296.4,11.0,316.0,7.0,221.9,11,3,1:04.269
15,16,Nicholas Latifi,Williams Racing,50,1:16:55.491,N. Latifi,Abu-dhabi,2021,15.0,1:29.293,...,328.3,10.0,294.1,9.0,321.8,14.0,218.8,6,1,21.241
16,17,Antonio Giovinazzi,Alfa Romeo Racing ORLEN,33,0:50:20.298,A. Giovinazzi,Abu-dhabi,2021,16.0,1:29.442,...,331.7,12.0,291.6,6.0,325.7,17.0,215.4,99,1,22.283
17,18,George Russell,Williams Racing,26,0:41:10.855,G. Russell,Abu-dhabi,2021,19.0,1:30.647,...,321.7,15.0,288.3,16.0,309.5,18.0,214.7,NaN,NaN,NaN


In [139]:
df_2021_session = fia_f1_session(2021, gp_city = city_21, gp_link = links_session_21)

Season: 2021 | Source: fia.com
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/bahrain-grand-prix/session-classifications
Circuit: emilia-romagna | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/emilia-romagna-grand-prix/session
Circuit: portuguese | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/portuguese-grand-prix/session-classifications
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/spanish-grand-prix/session-classifications
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/monaco-grand-prix/session-classifications
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/azerbaijan-grand-prix/session-classifications
Circuit: french | https://www.fia.com/events/fia-formula-one-world-championship/season-2021/french-grand-prix/session-classifications
Circuit: au

In [140]:
df_2021_session

,QL_CLASS,DRIVER,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,GRAND_PRIX,SEASON,GD_CLASS,GD_TIME
0,1,Max Verstappen,1:30.499,3,1:30.318,6.0,1:28.997,6.0,Bahrain,2021,1.0,1:28.997
1,2,Lewis Hamilton,1:30.617,6,1:30.085,6.0,1:29.385,6.0,Bahrain,2021,2.0,1:29.385
2,3,Valtteri Bottas,1:31.200,5,1:30.186,6.0,1:29.586,6.0,Bahrain,2021,3.0,1:29.586
3,4,Charles Leclerc,1:30.691,6,1:30.010,6.0,1:29.678,3.0,Bahrain,2021,4.0,1:29.678
4,5,Pierre Gasly,1:30.848,3,1:30.513,6.0,1:29.809,6.0,Bahrain,2021,5.0,1:29.809
...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Nicholas Latifi,1:24.338,8,NaN,NaN,NaN,NaN,Abu-dhabi,2021,16.0,1:24.338
16,17,George Russell,1:24.423,7,NaN,NaN,NaN,NaN,Abu-dhabi,2021,17.0,1:24.423
17,18,Kimi Raikkonen,1:24.779,8,NaN,NaN,NaN,NaN,Abu-dhabi,2021,18.0,1:24.779
18,19,Mick Schumacher,1:24.906,9,NaN,NaN,NaN,NaN,Abu-dhabi,2021,19.0,1:24.906


In [141]:
f1_2021 = pd.merge(df_2021, df_2021_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [142]:
f1_2021

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,Bahrain,2021,4.0,1:34.015,...,2.0,1:30.617,6.0,1:30.085,6.0,1:29.385,6.0,2021.0,2.0,1:29.385
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,Bahrain,2021,2.0,1:33.228,...,1.0,1:30.499,3.0,1:30.318,6.0,1:28.997,6.0,2021.0,1.0,1:28.997
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,Bahrain,2021,1.0,1:32.090,...,3.0,1:31.200,5.0,1:30.186,6.0,1:29.586,6.0,2021.0,3.0,1:29.586
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,Bahrain,2021,6.0,1:34.396,...,7.0,1:30.902,6.0,1:30.099,6.0,1:29.974,6.0,2021.0,7.0,1:29.974
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,Bahrain,2021,3.0,1:33.970,...,11.0,1:31.165,5.0,1:30.659,6.0,NaN,NaN,2021.0,11.0,1:30.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,15,Sergio Perez,Red Bull Racing Honda,55,1:24:12.571,S. Perez,Abu-dhabi,2021,2.0,1:26.419,...,4.0,1:23.350,9.0,1:23.135,8.0,1:22.947,7.0,2021.0,4.0,1:22.947
475,16,Nicholas Latifi,Williams Racing,50,1:16:55.491,N. Latifi,Abu-dhabi,2021,15.0,1:29.293,...,16.0,1:24.338,8.0,NaN,NaN,NaN,NaN,2021.0,16.0,1:24.338
476,17,Antonio Giovinazzi,Alfa Romeo Racing ORLEN,33,0:50:20.298,A. Giovinazzi,Abu-dhabi,2021,16.0,1:29.442,...,14.0,1:24.118,9.0,1:24.251,8.0,NaN,NaN,2021.0,14.0,1:24.251
477,18,George Russell,Williams Racing,26,0:41:10.855,G. Russell,Abu-dhabi,2021,19.0,1:30.647,...,17.0,1:24.423,7.0,NaN,NaN,NaN,NaN,2021.0,17.0,1:24.423


# 2020

In [143]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=1059', url_string = 'race')

['/events/karting/season-2022/races-calendar',
 '/race-against-manipulation',
 '/events/fia-formula-one-world-championship/season-2020/singapore-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/australian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/vietnamese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/dutch-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2020/canadian-grand-prix/race-classification',
 '/events/fia-formula-one-wor

In [144]:
links_race_20 = [
'/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/british-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/race',
'/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/italian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/tuscan-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/russian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/eifel-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/portuguese-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/emilia-romagna-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/turkish-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/race-classification-0',
'/events/fia-formula-one-world-championship/season-2020/sakhir-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/race-classification-0']

In [145]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=1059', url_string = 'session')

['/events/fia-formula-one-world-championship/season-2020/singapore-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/australian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/vietnamese-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/dutch-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/monaco-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/azerbaijan-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/canadian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2020/french-grand-p

In [146]:
links_session_20 = [
 '/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/british-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/session'
,'/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/italian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/tuscan-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/russian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/eifel-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/portuguese-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/emilia-romagna-grand-prix/session'
,'/events/fia-formula-one-world-championship/season-2020/turkish-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/session-classifications-0'
,'/events/fia-formula-one-world-championship/season-2020/sakhir-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/session-classifications-0']

In [147]:
c_20 = f1_gp_circuits(season = 2020)
city_20 = c_20['GP'].tolist()
c_20['GP'][4] = '70th-anniversary'
c_20

Season: 2020 | Source: espn.com


,Date,Race,GP
0,Jul 3 - 5,Austrian GPRed Bull Ring,austrian
1,Jul 10 - 12,Austrian GP 2Red Bull Ring,austrian
2,Jul 17 - 19,Hungarian GPHungaroring,hungarian
3,Jul 31 - Aug 2,Rolex British GPSilverstone Circuit,british
4,Aug 7 - 9,Rolex British GP 2Silverstone Circuit,70th-anniversary
5,Aug 14 - 16,Spanish GPCircuit de Barcelona-Catalunya,spanish
6,Aug 28 - 30,Belgian GPCircuit de Spa-Francorchamps,belgian
7,Sep 4 - 6,Italian GPAutodromo Nazionale Monza,italian
8,Sep 11 - 13,Tuscan GPAutodromo Internazionale del Mugello,tuscan
9,Sep 25 - 27,Russian GPSochi Autodrom,russian


In [148]:
df_2020 = fia_f1_data(2020, gp_city = city_20, gp_link = links_race_20)

Season: 2020 | Source: fia.com
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/race-classification
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/race-classification
Circuit: hungarian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/race-classification
Circuit: british | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/british-grand-prix/race-classification
Circuit: british | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/race
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/race-classification
Circuit: belgian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/race-classification
Circuit: italian | https://www.fia

In [149]:
df_2020

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,STKM/H,I1_POS,I1_KM/H,I2_POS,KM/H.1,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,71,1:30:55.739,V. Bottas,Austrian,2020,2.0,1:07.657,...,299.9,19.0,304.5,2.0,241.9,16.0,280.7,77,2,38.344
1,2,Charles Leclerc,Scuderia Ferrari,71,1:30:58.439,C. Leclerc,Austrian,2020,4.0,1:07.901,...,318.2,13.0,322.7,1.0,243.5,10.0,284.7,16,3,1:03.476
2,3,Lando Norris,McLaren F1 Team,71,1:31:01.230,L. Norris,Austrian,2020,1.0,1:07.475,...,313.2,12.0,323.1,7.0,239.5,4.0,287.7,4,3,1:03.721
3,4,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,71,1:31:01.428,L. Hamilton,Austrian,2020,3.0,1:07.712,...,324.0,9.0,326.0,4.0,240.5,3.0,287.7,44,2,38.215
4,5,Carlos Sainz,McLaren F1 Team,71,1:31:04.642,C. Sainz,Austrian,2020,5.0,1:07.974,...,326.5,2.0,334.8,8.0,238.5,1.0,291.4,55,3,1:05.696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Antonio Giovinazzi,Alfa Romeo Racing ORLEN,54,1:37:02.381,A. Giovinazzi,Abu-dhabi,2020,7.0,1:41.675,...,333.1,2.0,291.8,12.0,306.4,10.0,221.0,99,1,21.480
16,17,Nicholas Latifi,Williams Racing,54,1:37:11.562,N. Latifi,Abu-dhabi,2020,16.0,1:42.497,...,329.3,11.0,286.7,10.0,308.0,4.0,222.5,6,2,43.741
17,18,Kevin Magnussen,Haas F1 Team,54,1:38:00.387,K. Magnussen,Abu-dhabi,2020,13.0,1:41.999,...,311.2,16.0,285.0,18.0,299.8,12.0,220.4,20,2,45.812
18,19,Pietro Fittipaldi,Haas F1 Team,53,1:36:38.988,P. Fittipaldi,Abu-dhabi,2020,8.0,1:41.707,...,320.5,15.0,285.5,15.0,303.9,9.0,221.1,51,3,1:18.047


In [150]:
df_2020_session = fia_f1_session(2020, gp_city = city_20, gp_link = links_session_20)

Season: 2020 | Source: fia.com
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/session-classifications
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/session-classifications
Circuit: hungarian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/session-classifications
Circuit: british | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/british-grand-prix/session-classifications
Circuit: british | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/session
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/session-classifications
Circuit: belgian | https://www.fia.com/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/session-classifications
Circuit

In [151]:
df_2020_session

,QL_CLASS,DRIVER,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,GRAND_PRIX,SEASON,GD_CLASS,GD_TIME
0,1,Valtteri Bottas,1:04.111,6,1:03.015,6.0,1:02.939,7.0,Austrian,2020,1,1:02.939
1,2,Lewis Hamilton,1:04.198,8,1:03.096,6.0,1:02.951,7.0,Austrian,2020,5,1:02.951
2,3,Max Verstappen,1:04.024,8,1:04.000,8.0,1:03.477,7.0,Austrian,2020,2,1:03.477
3,4,Lando Norris,1:04.606,6,1:03.819,6.0,1:03.626,5.0,Austrian,2020,3,1:03.626
4,5,Alexander Albon,1:04.661,6,1:03.746,6.0,1:03.868,6.0,Austrian,2020,4,1:03.868
...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Kimi Raikkonen,1:37.555,6,NaN,NaN,NaN,NaN,Abu-dhabi,2020,15,1:37.555
16,17,Kevin Magnussen,1:37.863,9,NaN,NaN,NaN,NaN,Abu-dhabi,2020,20,1:37.863
17,18,George Russell,1:38.045,8,NaN,NaN,NaN,NaN,Abu-dhabi,2020,16,1:38.045
18,19,Pietro Fittipaldi,1:38.173,9,NaN,NaN,NaN,NaN,Abu-dhabi,2020,17,1:38.173


In [152]:
f1_2020 = pd.merge(df_2020, df_2020_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [153]:
f1_2020

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,71,1:30:55.739,V. Bottas,Austrian,2020,2.0,1:07.657,...,1,1:04.111,6,1:03.015,6.0,1:02.939,7.0,2020,1,1:02.939
1,1,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,71,1:30:55.739,V. Bottas,Austrian,2020,2.0,1:07.657,...,4,1:18.791,13,1:18.657,11.0,1:20.701,10.0,2020,4,1:20.701
2,2,Charles Leclerc,Scuderia Ferrari,71,1:30:58.439,C. Leclerc,Austrian,2020,4.0,1:07.901,...,7,1:04.500,8,1:04.041,6.0,1:03.923,6.0,2020,7,1:03.923
3,2,Charles Leclerc,Scuderia Ferrari,71,1:30:58.439,C. Leclerc,Austrian,2020,4.0,1:07.901,...,11,1:20.871,12,1:19.628,12.0,NaN,NaN,2020,14,1:19.628
4,3,Lando Norris,McLaren F1 Team,71,1:31:01.230,L. Norris,Austrian,2020,1.0,1:07.475,...,4,1:04.606,6,1:03.819,6.0,1:03.626,5.0,2020,3,1:03.626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,16,Antonio Giovinazzi,Alfa Romeo Racing ORLEN,54,1:37:02.381,A. Giovinazzi,Abu-dhabi,2020,7.0,1:41.675,...,14,1:37.075,6,1:38.248,6.0,NaN,NaN,2020,14,1:38.248
416,17,Nicholas Latifi,Williams Racing,54,1:37:11.562,N. Latifi,Abu-dhabi,2020,16.0,1:42.497,...,20,1:38.443,7,NaN,NaN,NaN,NaN,2020,18,1:38.443
417,18,Kevin Magnussen,Haas F1 Team,54,1:38:00.387,K. Magnussen,Abu-dhabi,2020,13.0,1:41.999,...,17,1:37.863,9,NaN,NaN,NaN,NaN,2020,20,1:37.863
418,19,Pietro Fittipaldi,Haas F1 Team,53,1:36:38.988,P. Fittipaldi,Abu-dhabi,2020,8.0,1:41.707,...,19,1:38.173,9,NaN,NaN,NaN,NaN,2020,17,1:38.173


# 2019

In [154]:
links_race_19 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=971', url_string = 'race-cl')

In [155]:
links_race_19 = ['/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/french-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/british-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/german-grand-prix/race-classification-german',
 '/events/fia-formula-one-world-championship/season-2019/hungarian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/belgian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/italian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/singapore-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/russian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/japanese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/united-states-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/brazilian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/abu-dhabi-grand-prix/race-classification']

In [156]:
links_session_19 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=971', url_string = 'session')

In [157]:
links_session_19 = ['/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/french-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/british-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/german-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/hungarian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/belgian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/italian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/singapore-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/russian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/japanese-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/united-states-grand-prix/session',
 '/events/fia-formula-one-world-championship/season-2019/brazilian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/abu-dhabi-grand-prix/session-classifications']

In [158]:
c_19 = f1_gp_circuits(season = 2019)
c_19 = c_19.drop(index=8)
c_19 = c_19.drop(index=17)
city_19 = c_19['GP'].unique().tolist()
c_19

Season: 2019 | Source: espn.com


,Date,Race,GP
0,Mar 15 - 17,Australian GPMelbourne Grand Prix Circuit,australian
1,Mar 29 - 31,Bahrain GPBahrain International Circuit,bahrain
2,Apr 12 - 14,Chinese GPShanghai International Circuit,chinese
3,Apr 26 - 28,Socar Azerbaijan GPBaku City Circuit,azerbaijan
4,May 10 - 12,Spanish GPCircuit de Barcelona-Catalunya,spanish
5,May 23 - 26,Monaco GPCircuit de Monaco,monaco
6,Jun 7 - 9,Canadian GPCircuit Gilles-Villeneuve,canadian
7,Jun 21 - 23,French GPCircuit Paul Ricard,french
9,Jul 12 - 14,Rolex British GPSilverstone Circuit,british
10,Jul 26 - 28,Mercedes-Benz German GPHockenheimring,german


In [159]:
df_2019 = fia_f1_data(2019, gp_city = city_19, gp_link = links_race_19)

Season: 2019 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/race-classification
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/race-classification
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/race-classification
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/azerbaijan-grand-prix/race-classification
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/race-classification
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/race-classification
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/race-classification
Circuit: french | https://www.fia

In [160]:
df_2019

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,STKM/H,I1_POS,I1_KM/H,I2_POS,KM/H.1,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Valtteri Bottas,Mercedes AMG Petronas Motorsport,58,1:25:27.325,V. Bottas,Australian,2019,1,1:25.580,...,311.3,3,280.3,9,290.9,11,301.4,77,1,22.014
1,2,Lewis Hamilton,Mercedes AMG Petronas Motorsport,58,1:25:48.211,L. Hamilton,Australian,2019,2,1:26.057,...,309.3,9,278.2,16,287.2,17,296.1,44,1,21.515
2,3,Max Verstappen,Aston Martin Red Bull Racing,58,1:25:49.845,M. Verstappen,Australian,2019,3,1:26.256,...,319.9,1,281.7,4,292.9,6,304.7,33,1,21.157
3,4,Sebastian Vettel,Scuderia Ferrari,58,1:26:24.434,S. Vettel,Australian,2019,8,1:27.954,...,303.7,13,274.6,19,283.1,19,291.2,5,1,21.995
4,5,Charles Leclerc,Scuderia Ferrari,58,1:26:25.555,C. Leclerc,Australian,2019,4,1:26.926,...,297.4,11,277.4,13,288.7,12,301.2,16,1,22.306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Antonio Giovinazzi,Alfa Romeo Racing,54,1:35:32.482,A. Giovinazzi,Abu-dhabi,2019,15,1:43.256,...,334.9,14,284.8,13,308.9,13,219.3,99,2,44.165
16,17,George Russell,ROKiT Williams Racing,54,1:35:43.499,G. Russell,Abu-dhabi,2019,13,1:43.074,...,329.3,11,286.7,12,309.1,14,219.1,63,1,22.272
17,18,Pierre Gasly,Red Bull Toro Rosso Honda,53,1:34:17.001,P. Gasly,Abu-dhabi,2019,10,1:42.414,...,327.7,5,290.7,3,324.4,5,223.0,10,1,1:17.467
18,19,Robert Kubica,ROKiT Williams Racing,53,1:34:29.473,R. Kubica,Abu-dhabi,2019,20,1:44.500,...,317.8,20,281.9,19,302.8,16,218.2,88,1,21.688


In [161]:
df_2019_session = fia_f1_session(2019, gp_city = city_19, gp_link = links_session_19)

Season: 2019 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/session-classifications
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/session-classifications
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/session-classifications
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/session-classifications
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/session-classifications
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/session-classifications
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2019/french-grand-prix/session-classifications
Circuit: 

In [162]:
f1_2019 = pd.merge(df_2019, df_2019_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [163]:
f1_2019

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Valtteri Bottas,Mercedes AMG Petronas Motorsport,58,1:25:27.325,V. Bottas,Australian,2019,1,1:25.580,...,2.0,1:22.367,7.0,1:21.193,6.0,1:20.598,6.0,2019.0,2.0,1:20.598
1,2,Lewis Hamilton,Mercedes AMG Petronas Motorsport,58,1:25:48.211,L. Hamilton,Australian,2019,2,1:26.057,...,1.0,1:22.043,6.0,1:21.014,6.0,1:20.486,6.0,2019.0,1.0,1:20.486
2,3,Max Verstappen,Aston Martin Red Bull Racing,58,1:25:49.845,M. Verstappen,Australian,2019,3,1:26.256,...,4.0,1:22.876,5.0,1:21.678,6.0,1:21.320,6.0,2019.0,4.0,1:21.320
3,4,Sebastian Vettel,Scuderia Ferrari,58,1:26:24.434,S. Vettel,Australian,2019,8,1:27.954,...,3.0,1:22.885,5.0,1:21.912,5.0,1:21.190,6.0,2019.0,3.0,1:21.190
4,5,Charles Leclerc,Scuderia Ferrari,58,1:26:25.555,C. Leclerc,Australian,2019,4,1:26.926,...,5.0,1:22.017,8.0,1:21.739,3.0,1:21.442,6.0,2019.0,5.0,1:21.442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,16,Antonio Giovinazzi,Alfa Romeo Racing,54,1:35:32.482,A. Giovinazzi,Abu-dhabi,2019,15,1:43.256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,17,George Russell,ROKiT Williams Racing,54,1:35:43.499,G. Russell,Abu-dhabi,2019,13,1:43.074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,18,Pierre Gasly,Red Bull Toro Rosso Honda,53,1:34:17.001,P. Gasly,Abu-dhabi,2019,10,1:42.414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,19,Robert Kubica,ROKiT Williams Racing,53,1:34:29.473,R. Kubica,Abu-dhabi,2019,20,1:44.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2018

In [164]:
links_race_18 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=866', url_string = 'race-class')

In [165]:
links_race_18 = ['/events/fia-formula-one-world-championship/season-2018/race-classification',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-25',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-23',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-5',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-6',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-8',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-24',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-26',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-27',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-28',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-13',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-29',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-30',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-22',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-17',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-18',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-20']

In [166]:
links_session_18 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=866', url_string = 'session')

In [167]:
links_session_18 = ['/events/fia-formula-one-world-championship/season-2018/session-classifications',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-25',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-23',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-5',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-6',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-24',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-26',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-27',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-28',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-13',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-29',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-30',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-22',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-17',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-18',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-19']

In [168]:
c_18 = f1_gp_circuits(season = 2018)
city_18 = c_18['GP'].unique().tolist()
c_18

Season: 2018 | Source: espn.com


,Date,Race,GP
0,Feb 23 - Mar 25,Australian GPMelbourne Grand Prix Circuit,australian
1,Apr 6 - 8,Bahrain GPBahrain International Circuit,bahrain
2,Apr 13 - 15,Chinese GPShanghai International Circuit,chinese
3,Apr 27 - 29,Azerbaijan GPBaku City Circuit,azerbaijan
4,May 11 - 13,Spanish GPCircuit de Barcelona-Catalunya,spanish
5,May 24 - 27,Monaco GPCircuit de Monaco,monaco
6,Jun 8 - 10,Canadian GPCircuit Gilles-Villeneuve,canadian
7,Jun 22 - 24,French GPCircuit Paul Ricard,french
8,Jun 29 - Jul 1,Austrian GPRed Bull Ring,austrian
9,Jul 6 - 8,British GPSilverstone Circuit,british


In [169]:
df_2018 = fia_f1_data(2018, gp_city = city_18, gp_link = links_race_18)

Season: 2018 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-0
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-1
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-25
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-3
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-23
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-5
Circuit: french | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/race-classification-6
Circuit: austrian | https://www.fia.com/even

In [170]:
df_2018_session = fia_f1_session(2018, gp_city = city_18, gp_link = links_session_18)

Season: 2018 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-0
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-1
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-25
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-3
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-23
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-5
Circuit: french | https://www.fia.com/events/fia-formula-one-world-championship/season-2018/session-classifications-6
Circuit: aus

In [171]:
f1_2018 = pd.merge(df_2018, df_2018_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [172]:
f1_2018

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Sebastian Vettel,Scuderia Ferrari,58,1:29:33.283,S. Vettel,Australian,2018,4.0,1:26.469,...,3.0,1:23.348,7.0,1:21.944,6.0,1:21.838,7.0,2018.0,3.0,1:21.838
1,2,Lewis Hamilton,Mercedes AMG Petronas Motorsport,58,1:29:38.319,L. Hamilton,Australian,2018,3.0,1:26.444,...,1.0,1:22.824,7.0,1:22.051,5.0,1:21.164,8.0,2018.0,1.0,1:21.164
2,3,Kimi Raikkonen,Scuderia Ferrari,58,1:29:39.592,K. Raikkonen,Australian,2018,2.0,1:26.373,...,2.0,1:23.096,5.0,1:22.507,5.0,1:21.828,7.0,2018.0,2.0,1:21.828
3,4,Daniel Ricciardo,Aston Martin Red Bull Racing,58,1:29:40.352,D. Ricciardo,Australian,2018,1.0,1:25.945,...,5.0,1:23.494,5.0,1:22.897,5.0,1:22.152,7.0,2018.0,8.0,1:22.152
4,5,Fernando Alonso,McLaren F1 Team,58,1:30:01.169,F. Alonso,Australian,2018,7.0,1:26.978,...,11.0,1:23.597,8.0,1:23.692,6.0,NaN,NaN,2018.0,10.0,1:23.692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,16,Pierre Gasly,Red Bull Toro Rosso Honda,46,1:26:09.517,P. Gasly,Abu-dhabi,2018,15.0,1:43.988,...,17.0,1:38.166,9.0,NaN,NaN,NaN,NaN,2018.0,17.0,1:38.166
416,17,Esteban Ocon,Racing Point Force India F1 Team,44,1:22:30.745,E. Ocon,Abu-dhabi,2018,13.0,1:43.591,...,9.0,1:36.936,6.0,1:36.814,6.0,1:36.540,6.0,2018.0,9.0,1:36.540
417,18,Marcus Ericsson,Alfa Romeo Sauber F1 Team,24,0:47:06.407,M. Ericsson,Abu-dhabi,2018,19.0,1:46.077,...,12.0,1:37.619,8.0,1:37.132,6.0,NaN,NaN,2018.0,12.0,1:37.132
418,19,Kimi Raikkonen,Scuderia Ferrari,6,0:14:06.737,K. Raikkonen,Abu-dhabi,2018,18.0,1:45.198,...,4.0,1:37.010,5.0,1:36.735,7.0,1:35.365,6.0,2018.0,4.0,1:35.365


# 2017

In [173]:
links_race_17 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=679', url_string = 'race-class')

In [174]:
links_race_17

['/events/fia-formula-one-world-championship/season-2017/race-classification',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-8',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-2',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-9',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-10',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-11',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-12',
 '/events/fia-formula-one-world-championship/season-2017/race-classification-4',
 '/events/fia-formula-one-w

In [175]:
links_session_17 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=679', url_string = 'session')

In [176]:
links_session_17

['/events/fia-formula-one-world-championship/season-2017/session-classifications',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-2',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-9',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-10',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-11',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2017/session-classifications-12',
 '/events/fia-formula-one-world-championship/season-2017/session

In [177]:
c_17 = f1_gp_circuits(season = 2017)
city_17 = c_17['GP'].unique().tolist()
c_17

Season: 2017 | Source: espn.com


,Date,Race,GP
0,Mar 24 - 26,Australian GPMelbourne Grand Prix Circuit,australian
1,Apr 7 - 9,Chinese GPShanghai International Circuit,chinese
2,Apr 14 - 16,Bahrain GPBahrain International Circuit,bahrain
3,Apr 28 - 30,Russian GPSochi Autodrom,russian
4,May 12 - 14,Spanish GPCircuit de Barcelona-Catalunya,spanish
5,May 25 - 28,Monaco GPCircuit de Monaco,monaco
6,Jun 9 - 11,Canadian GPCircuit Gilles-Villeneuve,canadian
7,Jun 23 - 25,Azerbaijan GPBaku City Circuit,azerbaijan
8,Jul 7 - 9,Austrian GPRed Bull Ring,austrian
9,Jul 14 - 16,British GPSilverstone Circuit,british


In [178]:
df_2017 = fia_f1_data(2017, gp_city = city_17, gp_link = links_race_17)

Season: 2017 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-7
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-0
Circuit: russian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-8
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-1
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-2
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-9
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-10
Circuit: austrian | https://www.fia.com/even

In [179]:
df_2017

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,STKM/H,I1_POS,I1_KM/H,I2_POS,KM/H.1,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Sebastian Vettel,Scuderia Ferrari,57,1:24:11.672,S. Vettel,Australian,2017,3.0,1:26.638,...,314.6,4.0,280.8,3.0,298.8,5.0,305.1,5,1,21.988
1,2,Lewis Hamilton,Mercedes AMG Petronas F1 Team,57,1:24:21.647,L. Hamilton,Australian,2017,6.0,1:27.033,...,318.0,2.0,283.9,1.0,300.0,2.0,308.3,44,1,21.709
2,3,Valtteri Bottas,Mercedes AMG Petronas F1 Team,57,1:24:22.922,V. Bottas,Australian,2017,2.0,1:26.593,...,316.3,1.0,284.9,5.0,297.9,6.0,304.9,77,1,21.440
3,4,Kimi Raikkonen,Scuderia Ferrari,57,1:24:34.065,K. Raikkonen,Australian,2017,1.0,1:26.538,...,304.3,11.0,277.2,13.0,293.2,14.0,297.7,7,1,22.033
4,5,Max Verstappen,Red Bull Racing,57,1:24:40.499,M. Verstappen,Australian,2017,5.0,1:26.964,...,315.3,3.0,281.6,2.0,299.4,9.0,302.8,33,1,22.208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Pierre Gasly,SCUDERIA TORO ROSSO,54,1:35:33.961,P. Gasly,Abu-dhabi,2017,16.0,1:43.844,...,316.2,20.0,278.5,18.0,300.5,16.0,218.6,10,1,21.519
16,17,Marcus Ericsson,Sauber F1 Team,54,1:35:36.526,M. Ericsson,Abu-dhabi,2017,15.0,1:43.567,...,325.7,14.0,284.4,7.0,307.3,18.0,218.0,9,1,22.176
17,18,Lance Stroll,Williams Martini Racing,54,1:35:44.704,L. Stroll,Abu-dhabi,2017,6.0,1:42.324,...,324.9,6.0,288.3,11.0,306.1,6.0,220.4,18,3,1:05.064
18,19,Carlos Sainz Jr.,Renault Sport F1 Team,31,0:54:20.683,C. Sainz Jr.,Abu-dhabi,2017,14.0,1:43.378,...,319.0,18.0,281.0,17.0,301.9,19.0,216.4,55,1,21.956


In [180]:
df_2017_session = fia_f1_session(2017, gp_city = city_17, gp_link = links_session_17)

Season: 2017 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-7
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-0
Circuit: russian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-8
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-1
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-2
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-9
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/session-classifications-10
Circuit: aus

In [181]:
df_2017_session

,QL_CLASS,DRIVER,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,GRAND_PRIX,SEASON,GD_CLASS,GD_TIME
0,1,Lewis Hamilton,1:24.191,5,1:23.251,3.0,1:22.188,6.0,Australian,2017,1.0,1:22.188
1,2,Sebastian Vettel,1:25.210,5,1:23.401,6.0,1:22.456,6.0,Australian,2017,2.0,1:22.456
2,3,Valtteri Bottas,1:24.514,4,1:23.215,3.0,1:22.481,6.0,Australian,2017,3.0,1:22.481
3,4,Kimi Raikkonen,1:24.352,10,1:23.376,3.0,1:23.033,6.0,Australian,2017,4.0,1:23.033
4,5,Max Verstappen,1:24.482,7,1:24.092,6.0,1:23.485,5.0,Australian,2017,5.0,1:23.485
...,...,...,...,...,...,...,...,...,...,...,...,...
15,16,Romain Grosjean,1:39.516,10,NaN,NaN,NaN,NaN,Abu-dhabi,2017,16.0,1:39.516
16,17,Pierre Gasly,1:39.724,9,NaN,NaN,NaN,NaN,Abu-dhabi,2017,17.0,1:39.724
17,18,Pascal Wehrlein,1:39.930,9,NaN,NaN,NaN,NaN,Abu-dhabi,2017,18.0,1:39.930
18,19,Marcus Ericsson,1:39.994,9,NaN,NaN,NaN,NaN,Abu-dhabi,2017,19.0,1:39.994


In [182]:
f1_2017 = pd.merge(df_2017, df_2017_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [183]:
f1_2017

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Sebastian Vettel,Scuderia Ferrari,57,1:24:11.672,S. Vettel,Australian,2017,3.0,1:26.638,...,2.0,1:25.210,5.0,1:23.401,6.0,1:22.456,6.0,2017.0,2.0,1:22.456
1,2,Lewis Hamilton,Mercedes AMG Petronas F1 Team,57,1:24:21.647,L. Hamilton,Australian,2017,6.0,1:27.033,...,1.0,1:24.191,5.0,1:23.251,3.0,1:22.188,6.0,2017.0,1.0,1:22.188
2,3,Valtteri Bottas,Mercedes AMG Petronas F1 Team,57,1:24:22.922,V. Bottas,Australian,2017,2.0,1:26.593,...,3.0,1:24.514,4.0,1:23.215,3.0,1:22.481,6.0,2017.0,3.0,1:22.481
3,4,Kimi Raikkonen,Scuderia Ferrari,57,1:24:34.065,K. Raikkonen,Australian,2017,1.0,1:26.538,...,4.0,1:24.352,10.0,1:23.376,3.0,1:23.033,6.0,2017.0,4.0,1:23.033
4,5,Max Verstappen,Red Bull Racing,57,1:24:40.499,M. Verstappen,Australian,2017,5.0,1:26.964,...,5.0,1:24.482,7.0,1:24.092,6.0,1:23.485,5.0,2017.0,5.0,1:23.485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,16,Pierre Gasly,SCUDERIA TORO ROSSO,54,1:35:33.961,P. Gasly,Abu-dhabi,2017,16.0,1:43.844,...,17.0,1:39.724,9.0,NaN,NaN,NaN,NaN,2017.0,17.0,1:39.724
396,17,Marcus Ericsson,Sauber F1 Team,54,1:35:36.526,M. Ericsson,Abu-dhabi,2017,15.0,1:43.567,...,19.0,1:39.994,9.0,NaN,NaN,NaN,NaN,2017.0,19.0,1:39.994
397,18,Lance Stroll,Williams Martini Racing,54,1:35:44.704,L. Stroll,Abu-dhabi,2017,6.0,1:42.324,...,15.0,1:39.503,8.0,1:39.646,6.0,NaN,NaN,2017.0,15.0,1:39.646
398,19,Carlos Sainz Jr.,Renault Sport F1 Team,31,0:54:20.683,C. Sainz Jr.,Abu-dhabi,2017,14.0,1:43.378,...,12.0,1:38.810,3.0,1:38.725,6.0,NaN,NaN,2017.0,12.0,1:38.725


# 2016

In [184]:
links_race_16 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=644', url_string = 'race-class')

In [185]:
links_race_16

['/events/fia-formula-one-world-championship/season-2016/race-classification',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-2',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-4',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-5',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-6',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-8',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-9',
 '/events/fia-formula-one-world-championship/season-2016/race-classification-10',
 '/events/fia-formula-one-wor

In [186]:
links_session_16 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=644', url_string = 'session')

In [187]:
links_session_16

['/events/fia-formula-one-world-championship/season-2016/session-classifications',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-2',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-4',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-5',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-6',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2016/session-classifications-9',
 '/events/fia-formula-one-world-championship/season-2016/session-cl

In [188]:
c_16 = f1_gp_circuits(season = 2016)
city_16 = c_16['GP'].unique().tolist()
c_16

Season: 2016 | Source: espn.com


,Date,Race,GP
0,Mar 18 - 20,Australian GPMelbourne Grand Prix Circuit,australian
1,Apr 1 - 3,Bahrain GPBahrain International Circuit,bahrain
2,Apr 15 - 17,Chinese GPShanghai International Circuit,chinese
3,Apr 29 - May 1,Russian GPSochi Autodrom,russian
4,May 13 - 15,Spanish GPCircuit de Barcelona-Catalunya,spanish
5,May 26 - 29,Monaco GPCircuit de Monaco,monaco
6,Jun 10 - 12,Canadian GPCircuit Gilles-Villeneuve,canadian
7,Jun 17 - 19,European GP,european
8,Jul 1 - 3,Austrian GPRed Bull Ring,austrian
9,Jul 8 - 10,British GPSilverstone Circuit,british


In [189]:
df_2016 = fia_f1_data(2016, gp_city = city_16, gp_link = links_race_16)

Season: 2016 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-0
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-1
Circuit: russian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-2
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-3
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-4
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-5
Circuit: european | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/race-classification-6
Circuit: austrian | https://www.fia.com/events/

In [190]:
df_2016

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,STKM/H,I1_POS,I1_KM/H,I2_POS,KM/H.1,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Nico Rosberg,Mercedes AMG Petronas F1 Team,57,1:48:15.565,N. Rosberg,Australian,2016,3.0,1:30.557,...,315.6,4.0,277.9,6.0,301.3,15.0,302.9,6,2,18:30.834
1,2,Lewis Hamilton,Mercedes AMG Petronas F1 Team,57,1:48:23.625,L. Hamilton,Australian,2016,4.0,1:30.646,...,314.8,1.0,278.7,1.0,305.2,2.0,309.2,44,2,18:32.027
2,3,Sebastian Vettel,Scuderia Ferrari,57,1:48:25.208,S. Vettel,Australian,2016,2.0,1:29.951,...,312.1,8.0,276.5,5.0,301.9,3.0,308.8,5,3,18:55.199
3,4,Daniel Ricciardo,Red Bull Racing,57,1:48:39.895,D. Ricciardo,Australian,2016,1.0,1:28.997,...,309.7,3.0,278.4,15.0,295.0,1.0,309.8,3,3,18:54.048
4,5,Felipe Massa,Williams Martini Racing,57,1:49:14.544,F. Massa,Australian,2016,9.0,1:32.288,...,308.2,9.0,274.8,8.0,298.0,13.0,303.2,19,2,18:31.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,18,Carlos Sainz Jr.,Scuderia Toro Rosso,41,1:15:22.129,C. Sainz Jr.,Abu-dhabi,2016,18.0,1:46.591,...,323.6,21.0,275.1,19.0,299.7,17.0,216.9,55,2,43.696
18,19,Daniil Kvyat,Scuderia Toro Rosso,14,0:26:02.917,D. Kvyat,Abu-dhabi,2016,21.0,1:48.752,...,326.8,22.0,271.4,15.0,303.1,21.0,213.6,26,1,21.896
19,20,Jenson Button,McLaren Honda,12,0:22:42.839,J. Button,Abu-dhabi,2016,22.0,1:48.753,...,316.0,17.0,284.3,21.0,298.2,20.0,214.0,NaN,NaN,NaN
20,21,Valtteri Bottas,Williams Martini Racing,6,0:11:03.071,V. Bottas,Abu-dhabi,2016,19.0,1:47.837,...,316.9,20.0,277.8,10.0,309.4,13.0,217.6,NaN,NaN,NaN


In [191]:
df_2016_session = fia_f1_session(2016, gp_city = city_16, gp_link = links_session_16)

Season: 2016 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-0
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-1
Circuit: russian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-2
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-3
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-4
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-5
Circuit: european | https://www.fia.com/events/fia-formula-one-world-championship/season-2016/session-classifications-6
Circuit: austri

In [192]:
f1_2016 = pd.merge(df_2016, df_2016_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [193]:
f1_2016

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Nico Rosberg,Mercedes AMG Petronas F1 Team,57,1:48:15.565,N. Rosberg,Australian,2016,3.0,1:30.557,...,2.0,1:26.934,4.0,1:24.796,3.0,1:24.197,6.0,2016.0,2.0,1:24.197
1,2,Lewis Hamilton,Mercedes AMG Petronas F1 Team,57,1:48:23.625,L. Hamilton,Australian,2016,4.0,1:30.646,...,1.0,1:25.351,5.0,1:24.605,3.0,1:23.837,6.0,2016.0,1.0,1:23.837
2,3,Sebastian Vettel,Scuderia Ferrari,57,1:48:25.208,S. Vettel,Australian,2016,2.0,1:29.951,...,3.0,1:26.945,5.0,1:25.257,6.0,1:24.675,3.0,2016.0,3.0,1:24.675
3,4,Daniel Ricciardo,Red Bull Racing,57,1:48:39.895,D. Ricciardo,Australian,2016,1.0,1:28.997,...,8.0,1:26.945,6.0,1:25.599,6.0,1:25.589,3.0,2016.0,8.0,1:25.589
4,5,Felipe Massa,Williams Martini Racing,57,1:49:14.544,F. Massa,Australian,2016,9.0,1:32.288,...,6.0,1:25.918,6.0,1:25.644,3.0,1:25.458,3.0,2016.0,6.0,1:25.458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,18,Carlos Sainz Jr.,Scuderia Toro Rosso,41,1:15:22.129,C. Sainz Jr.,Abu-dhabi,2016,18.0,1:46.591,...,21.0,1:42.393,9.0,NaN,NaN,NaN,NaN,2016.0,21.0,1:42.393
458,19,Daniil Kvyat,Scuderia Toro Rosso,14,0:26:02.917,D. Kvyat,Abu-dhabi,2016,21.0,1:48.752,...,17.0,1:42.003,9.0,NaN,NaN,NaN,NaN,2016.0,17.0,1:42.003
459,20,Jenson Button,McLaren Honda,12,0:22:42.839,J. Button,Abu-dhabi,2016,22.0,1:48.753,...,12.0,1:41.158,8.0,1:41.272,6.0,NaN,NaN,2016.0,12.0,1:41.272
460,21,Valtteri Bottas,Williams Martini Racing,6,0:11:03.071,V. Bottas,Abu-dhabi,2016,19.0,1:47.837,...,11.0,1:41.192,3.0,1:41.084,6.0,NaN,NaN,2016.0,11.0,1:41.084


# 2015

In [194]:
links_race_15 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=249', url_string = 'race-class')

In [195]:
links_race_15 = ['/events/fia-formula-one-world-championship/season-2015/race-classification',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-2',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-4',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-5',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-6',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-9',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-10',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-11',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-12',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-13',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-14',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-15',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-16',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-17',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-18']

In [196]:
links_session_15 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=249', url_string = 'session')

In [197]:
links_session_15 = ['/events/fia-formula-one-world-championship/season-2015/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-2',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-4',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-5',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-6',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-10',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-11',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-12',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-13',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-14',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-15',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-16',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-17',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-18',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications']

In [198]:
c_15 = f1_gp_circuits(season = 2015)
city_15 = c_15['GP'].unique().tolist()
c_15

Season: 2015 | Source: espn.com


,Date,Race,GP
0,Mar 13 - 15,Australian GPMelbourne Grand Prix Circuit,australian
1,Mar 27 - 29,Malaysian GPSepang International Circuit,malaysian
2,Apr 10 - 12,Chinese GPShanghai International Circuit,chinese
3,Apr 17 - 19,Bahrain GPBahrain International Circuit,bahrain
4,May 8 - 10,Spanish GPCircuit de Barcelona-Catalunya,spanish
5,May 21 - 24,Monaco GPCircuit de Monaco,monaco
6,Jun 5 - 7,Canadian GPCircuit Gilles-Villeneuve,canadian
7,Jun 19 - 21,Austrian GPRed Bull Ring,austrian
8,Jul 3 - 5,British GPSilverstone Circuit,british
9,Jul 24 - 26,Hungarian GPHungaroring,hungarian


In [199]:
df_2015 = fia_f1_data(2015, gp_city = city_15, gp_link = links_race_15)

Season: 2015 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification
Circuit: malaysian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-0
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-1
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-2
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-3
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-4
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-5
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/race-classification-6
Circuit: british | https://www.fia.com/events

In [200]:
df_2015_session = fia_f1_session(2015, gp_city = city_15, gp_link = links_session_15)

Season: 2015 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-0
Circuit: malaysian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-1
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-2
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-3
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-4
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-5
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-6
Circuit: austrian | https://www.fia.com/events/fia-formula-one-world-championship/season-2015/session-classifications-7
Circuit: br

In [201]:
f1_2015 = pd.merge(df_2015, df_2015_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [202]:
data = f1_2021.append(f1_2020, ignore_index = True)
data = data.append(f1_2019, ignore_index = True)
data = data.append(f1_2018, ignore_index = True)
data = data.append(f1_2017, ignore_index = True)
data = data.append(f1_2016, ignore_index = True)
data = data.append(f1_2015, ignore_index = True)

In [203]:
print('Shape: ' + str(data.shape))
data.head()

Shape: (2940, 41)


,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,Bahrain,2021,4.0,1:34.015,...,2.0,1:30.617,6.0,1:30.085,6.0,1:29.385,6.0,2021.0,2.0,1:29.385
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,Bahrain,2021,2.0,1:33.228,...,1.0,1:30.499,3.0,1:30.318,6.0,1:28.997,6.0,2021.0,1.0,1:28.997
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,Bahrain,2021,1.0,1:32.090,...,3.0,1:31.200,5.0,1:30.186,6.0,1:29.586,6.0,2021.0,3.0,1:29.586
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,Bahrain,2021,6.0,1:34.396,...,7.0,1:30.902,6.0,1:30.099,6.0,1:29.974,6.0,2021.0,7.0,1:29.974
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,Bahrain,2021,3.0,1:33.970,...,11.0,1:31.165,5.0,1:30.659,6.0,NaN,NaN,2021.0,11.0,1:30.659


In [204]:
#data.to_csv('f1_data_2.csv')

# Pre Processing

Update the pilots names to be consistent throught the dataset

In [205]:
df = data.copy()

In [206]:
df['DRIVER'] = df['DRIVER'].str.replace('Sergio Perez Mendoza','Sergio Perez')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('S. Perez Mendoza','S. Perez')

C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/1881939841.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('S. Perez Mendoza','S. Perez')


In [207]:
df['DRIVER'] = df['DRIVER'].str.replace(r'[^\x00-\x7f]', '')
df['DRIVER'] = df['DRIVER'].str.replace('Esteban Gutirrez','Esteban Gutierrez')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace(r'[^\x00-\x7f]', '')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('E. Gutirrez','E. Gutierrez')

C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/3815544717.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER'] = df['DRIVER'].str.replace(r'[^\x00-\x7f]', '')
C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/3815544717.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace(r'[^\x00-\x7f]', '')
C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/3815544717.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('E. Gutirrez','E. Gutierrez')


In [208]:
df['DRIVER'] = df['DRIVER'].str.replace('Carlos Sainz Jr.','Carlos Sainz')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('C. Sainz Jr.','C. Sainz')

C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/4039156399.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER'] = df['DRIVER'].str.replace('Carlos Sainz Jr.','Carlos Sainz')
C:\Users\nunoj\AppData\Local\Temp/ipykernel_4428/4039156399.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('C. Sainz Jr.','C. Sainz')


In [209]:
#df.to_csv('f1_prep.csv')

In [210]:
file = 'race_status.csv'
race_status = pd.read_csv(file, sep = ',')

In [211]:
race_status.head()

,DRIVER,GRAND_PRIX,SEASON,STATUS_ID,DRIVER_STATUS
0,Lewis Hamilton,australian,2015,1,Finished
1,Lewis Hamilton,malaysian,2015,1,Finished
2,Lewis Hamilton,chinese,2015,1,Finished
3,Lewis Hamilton,bahrain,2015,1,Finished
4,Lewis Hamilton,spanish,2015,1,Finished


In [212]:
df['GRAND_PRIX'] = df['GRAND_PRIX'].str.lower()


In [213]:
df.drop(['SEASON_y'], axis = 1)

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON_x,FLAP_POS,FLAP_TIME,...,PS_TOTAL_TIME,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,GD_CLASS,GD_TIME
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,bahrain,2021,4.0,1:34.015,...,48.915,2.0,1:30.617,6.0,1:30.085,6.0,1:29.385,6.0,2.0,1:29.385
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,bahrain,2021,2.0,1:33.228,...,48.615,1.0,1:30.499,3.0,1:30.318,6.0,1:28.997,6.0,1.0,1:28.997
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,bahrain,2021,1.0,1:32.090,...,1:21.725,3.0,1:31.200,5.0,1:30.186,6.0,1:29.586,6.0,3.0,1:29.586
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,bahrain,2021,6.0,1:34.396,...,50.539,7.0,1:30.902,6.0,1:30.099,6.0,1:29.974,6.0,7.0,1:29.974
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,bahrain,2021,3.0,1:33.970,...,1:12.289,11.0,1:31.165,5.0,1:30.659,6.0,NaN,NaN,11.0,1:30.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,16,Max Verstappen,Scuderia Toro Rosso,54,1:39:09.064,M. Verstappen,abu-dhabi,2015,6.0,1:45.746,...,1:07.121,11.0,1:42.889,8.0,1:42.521,6.0,NaN,NaN,11.0,1:42.521
2936,17,Fernando Alonso,McLaren Honda,53,1:39:17.721,F. Alonso,abu-dhabi,2015,3.0,1:44.796,...,1:45.741,17.0,1:43.187,5.0,NaN,NaN,NaN,NaN,16.0,1:43.187
2937,18,Will Stevens,Manor Marussia F1 Team,53,1:39:54.204,W. Stevens,abu-dhabi,2015,18.0,1:49.610,...,45.767,19.0,1:46.297,7.0,NaN,NaN,NaN,NaN,19.0,1:46.297
2938,19,Roberto Merhi Muntan,Manor Marussia F1 Team,52,1:39:11.622,R. Merhi Muntan,abu-dhabi,2015,19.0,1:51.213,...,24.314,20.0,1:47.434,8.0,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
df.rename(columns= {'SEASON_x':'SEASON'}, inplace=True)

In [215]:
df.head()

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,QL_CLASS,QL_TIME1,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,bahrain,2021,4.0,1:34.015,...,2.0,1:30.617,6.0,1:30.085,6.0,1:29.385,6.0,2021.0,2.0,1:29.385
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,bahrain,2021,2.0,1:33.228,...,1.0,1:30.499,3.0,1:30.318,6.0,1:28.997,6.0,2021.0,1.0,1:28.997
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,bahrain,2021,1.0,1:32.090,...,3.0,1:31.200,5.0,1:30.186,6.0,1:29.586,6.0,2021.0,3.0,1:29.586
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,bahrain,2021,6.0,1:34.396,...,7.0,1:30.902,6.0,1:30.099,6.0,1:29.974,6.0,2021.0,7.0,1:29.974
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,bahrain,2021,3.0,1:33.970,...,11.0,1:31.165,5.0,1:30.659,6.0,NaN,NaN,2021.0,11.0,1:30.659


In [216]:
df_new = pd.merge(df, race_status, how='left', left_on=['DRIVER','GRAND_PRIX','SEASON'], right_on = ['DRIVER','GRAND_PRIX','SEASON'])

In [217]:
df_new

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,...,QL_LAPS1,QL_TIME2,QL_LAPS2,QL_TIME3,QL_LAPS3,SEASON_y,GD_CLASS,GD_TIME,STATUS_ID,DRIVER_STATUS
0,1,Lewis Hamilton,Mercedes-AMG Petronas F1 Team,56,1:32:03.897,L. Hamilton,bahrain,2021,4.0,1:34.015,...,6.0,1:30.085,6.0,1:29.385,6.0,2021.0,2.0,1:29.385,1.0,Finished
1,2,Max Verstappen,Red Bull Racing Honda,56,1:32:04.642,M. Verstappen,bahrain,2021,2.0,1:33.228,...,3.0,1:30.318,6.0,1:28.997,6.0,2021.0,1.0,1:28.997,1.0,Finished
2,3,Valtteri Bottas,Mercedes-AMG Petronas F1 Team,56,1:32:41.280,V. Bottas,bahrain,2021,1.0,1:32.090,...,5.0,1:30.186,6.0,1:29.586,6.0,2021.0,3.0,1:29.586,1.0,Finished
3,4,Lando Norris,McLaren F1 Team,56,1:32:50.363,L. Norris,bahrain,2021,6.0,1:34.396,...,6.0,1:30.099,6.0,1:29.974,6.0,2021.0,7.0,1:29.974,1.0,Finished
4,5,Sergio Perez,Red Bull Racing Honda,56,1:32:55.944,S. Perez,bahrain,2021,3.0,1:33.970,...,5.0,1:30.659,6.0,NaN,NaN,2021.0,11.0,1:30.659,1.0,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,16,Max Verstappen,Scuderia Toro Rosso,54,1:39:09.064,M. Verstappen,abu-dhabi,2015,6.0,1:45.746,...,8.0,1:42.521,6.0,NaN,NaN,2015.0,11.0,1:42.521,11.0,+1 Lap
2936,17,Fernando Alonso,McLaren Honda,53,1:39:17.721,F. Alonso,abu-dhabi,2015,3.0,1:44.796,...,5.0,NaN,NaN,NaN,NaN,2015.0,16.0,1:43.187,12.0,+2 Laps
2937,18,Will Stevens,Manor Marussia F1 Team,53,1:39:54.204,W. Stevens,abu-dhabi,2015,18.0,1:49.610,...,7.0,NaN,NaN,NaN,NaN,2015.0,19.0,1:46.297,12.0,+2 Laps
2938,19,Roberto Merhi Muntan,Manor Marussia F1 Team,52,1:39:11.622,R. Merhi Muntan,abu-dhabi,2015,19.0,1:51.213,...,8.0,NaN,NaN,NaN,NaN,2015.0,NaN,NaN,13.0,+3 Laps


In [219]:
df_new.to_csv('f1_fulldata_under_revision.csv')